In [ ]:
import torch
import torch.nn as nn

In [ ]:
import torch
import torch.nn as nn

# Define the basic residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(residual)
        out = self.relu(out)
        return out

# Define the ResNet model
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Example usage
resnet_model = ResNet(ResidualBlock, [3, 4, 6, 3], num_classes=1000)  # ResNet50 configuration
print(resnet_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [ ]:
import os
import pandas as pd

data_dir = '/content/drive/MyDrive/Classroom/ML Dataset'
filepaths = []
labels = []

folds = os.listdir(data_dir)
for fold in folds:
    foldpath = os.path.join(data_dir, fold)
    filelist = os.listdir(foldpath)
    for file in filelist:
        fpath = os.path.join(foldpath, file)
        filepaths.append(fpath)
        labels.append(fold)

# Concatenate data paths with labels into one dataframe
Fseries = pd.Series(filepaths, name= 'filepaths')
Lseries = pd.Series(labels, name='labels')
df = pd.concat([Fseries, Lseries], axis= 1)

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'data' is your dataset
# 'labels' are corresponding labels or targets
# Set random_state to ensure reproducibility if needed

labels=df.labels;

train_data, val_data, train_labels, val_labels = train_test_split(
    df, labels, test_size=0.2, random_state=42
)

# Now, train_data and train_labels contain 80% of the data for training,
# and val_data and val_labels contain 20% of the data for validation.


In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.preprocessing import LabelEncoder

class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

        # Initialize label encoder
        self.label_encoder = LabelEncoder()
        self.dataframe['labels'] = self.label_encoder.fit_transform(self.dataframe['labels'])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]

        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, label

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create DataLoader instances
train_dataset = CustomDataset(train_data, transform=transform)
val_dataset = CustomDataset(val_data, transform=transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate ResNet model
resnet_model = ResNet(ResidualBlock, [3, 4, 6, 3], num_classes=4).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)  # You can adjust the learning rate

# Training loop
num_epochs = 2  # Adjust as needed
for epoch in range(num_epochs):
    resnet_model.train()

    for inputs, labels in train_loader:
       inputs = inputs.to(device)
       labels = labels.to(device)
       optimizer.zero_grad()
       outputs = resnet_model(inputs)
       loss = criterion(outputs, labels)
       loss.backward()
       optimizer.step()


    # Validation loop
    resnet_model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = resnet_model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {accuracy:.4f}")

# Save the trained model if needed
torch.save(resnet_model.state_dict(), "resnet_model.pth")


Epoch 1/2, Validation Accuracy: 0.2879
Epoch 2/2, Validation Accuracy: 0.2576


In [ ]:
# Assuming you have a test dataset similar to the training and validation datasets
test_dataset = CustomDataset(val_data, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Set the model to evaluation mode
resnet_model.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = resnet_model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy:.4f}")



Test Accuracy: 0.2576


In [ ]:
# # Assuming you have a file path to a test image
# image_path = "/content/20231129_091122.jpg"

# # Load the image
# image = Image.open(image_path).convert("RGB")

# # Apply transformations (same as used during training)
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# input_image = transform(image)
# input_image = input_image.unsqueeze(0)  # Add batch dimension

# # Set the model to evaluation mode
# resnet_model.eval()

# # Perform inference
# with torch.no_grad():
#     input_image = input_image.to(device)
#     output = resnet_model(input_image)

# # Get predicted class
# _, predicted_class = torch.max(output, 1)
# # Use the original labels from your dataset
# original_labels = val_data.iloc[0, 1]

# print(f"Predicted Class: {predicted_class.item()}, Original Class: {original_labels}")



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate ResNet model
resnet_model = ResNet(ResidualBlock, [3, 4, 6, 3], num_classes=4).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)  # You can adjust the learning rate

# Training loop
num_epochs = 15  # Adjust as needed
for epoch in range(num_epochs):
    resnet_model.train()

    for inputs, labels in train_loader:
       inputs = inputs.to(device)
       labels = labels.to(device)
       optimizer.zero_grad()
       outputs = resnet_model(inputs)
       loss = criterion(outputs, labels)
       loss.backward()
       optimizer.step()

    # Validation loop
    resnet_model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = resnet_model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {accuracy:.4f}")

# Save the trained model if needed
torch.save(resnet_model.state_dict(), "resnet_model2.pth")


Epoch 1/15, Validation Accuracy: 0.3030
Epoch 2/15, Validation Accuracy: 0.3485
Epoch 3/15, Validation Accuracy: 0.3636
Epoch 4/15, Validation Accuracy: 0.4545
Epoch 5/15, Validation Accuracy: 0.4242
Epoch 6/15, Validation Accuracy: 0.5909
Epoch 7/15, Validation Accuracy: 0.6667
Epoch 8/15, Validation Accuracy: 0.6818
Epoch 10/15, Validation Accuracy: 0.6212
Epoch 11/15, Validation Accuracy: 0.5000
Epoch 12/15, Validation Accuracy: 0.6818
Epoch 13/15, Validation Accuracy: 0.4848
Epoch 15/15, Validation Accuracy: 0.6667


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate ResNet model
resnet_model = ResNet(ResidualBlock, [3, 4, 6, 3], num_classes=4).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.0001)  # You can adjust the learning rate

# Training loop
num_epochs = 20  # Adjust as needed
for epoch in range(num_epochs):
    resnet_model.train()

    for inputs, labels in train_loader:
       inputs = inputs.to(device)
       labels = labels.to(device)
       optimizer.zero_grad()
       outputs = resnet_model(inputs)
       loss = criterion(outputs, labels)
       loss.backward()
       optimizer.step()


    # Validation loop
    resnet_model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = resnet_model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {accuracy:.4f}")

# Save the trained model if needed
torch.save(resnet_model.state_dict(), "resnet_model3.pth")


Epoch 1/20, Validation Accuracy: 0.3030
Epoch 2/20, Validation Accuracy: 0.3030
Epoch 3/20, Validation Accuracy: 0.3030
Epoch 4/20, Validation Accuracy: 0.3333
Epoch 5/20, Validation Accuracy: 0.4394
Epoch 6/20, Validation Accuracy: 0.5909
Epoch 7/20, Validation Accuracy: 0.5606
Epoch 8/20, Validation Accuracy: 0.6515
Epoch 9/20, Validation Accuracy: 0.6818
Epoch 10/20, Validation Accuracy: 0.5758
Epoch 11/20, Validation Accuracy: 0.6515
Epoch 12/20, Validation Accuracy: 0.5303
Epoch 13/20, Validation Accuracy: 0.5000
Epoch 14/20, Validation Accuracy: 0.6364
Epoch 15/20, Validation Accuracy: 0.6364
Epoch 16/20, Validation Accuracy: 0.6515
Epoch 17/20, Validation Accuracy: 0.6364
